In [1]:
import os
import json
import pandas as pd
import traceback

In [6]:
import langchain_openai
from langchain_openai import ChatOpenAI


In [9]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
KEY=os.getenv("OPENAI_API_KEY")


In [12]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [13]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11dc38710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11e549ac0>, root_client=<openai.OpenAI object at 0x11d71da00>, root_async_client=<openai.AsyncOpenAI object at 0x11e49a8d0>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [15]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [16]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [17]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [19]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/var/folders/hm/5z1ct5rn1ts5_kqmggzhb1nh0000gn/T/ipykernel_48206/2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [22]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [23]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [24]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [25]:
file_path=r"/Users/mac/mcqgene/data.txt"

In [26]:
file_path


'/Users/mac/mcqgene/data.txt'

In [27]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [29]:
print(TEXT)

Machine learning (ML) is a field of Artificial Intelligence (AI) that enables systems to learn from data, identify patterns, and make decisions with minimal human intervention, rather than being explicitly programmed. It uses algorithms to analyze large datasets, improving performance as it is exposed to more data over time. Key applications include product recommendations, fraud detection, language translation, and optimizing supply chains. 
How It Works
1. Data Collection & Preprocessing:
Large datasets are collected and prepared for the algorithms. This involves cleaning the data (correcting errors, removing duplicates) and transforming it into a suitable, consistent format. 
2. Model Training:
Algorithms are trained on this data to identify patterns and relationships. The more data the model processes, the better it gets at its task. 
3. Model Output:
The result of training is a machine learning model that can be used to make predictions or classifications on new, unseen data. 
Typ

In [30]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [32]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [33]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/var/folders/hm/5z1ct5rn1ts5_kqmggzhb1nh0000gn/T/ipykernel_48206/347235031.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of Artificial Intelligence (AI) that enables systems to learn from data, identify patterns, and make decisions with minimal human intervention, rather than being explicitly programmed. It uses algorithms to analyze large datasets, improving performance as it is exposed to more data over time. Key applications include product recommendations, fraud detection, language translation, and optimizing supply chains. 
How It Works
1. Data Collection & Preprocessing:
Large datasets are collected and prepared for the algorithms. This involves cleaning the data (correcting errors, removing duplicates) and transforming it into a suitable, consistent format. 
2. Model Training:
Algorithms are trained on this data to identify patterns and relationships. The more data the model processes, the better it gets at its task. 
3. Model Output:
The result of training i

In [34]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1608
Prompt Tokens:1166
Completion Tokens:442
Total Cost:0.001246


In [35]:
response


{'text': 'Machine learning (ML) is a field of Artificial Intelligence (AI) that enables systems to learn from data, identify patterns, and make decisions with minimal human intervention, rather than being explicitly programmed. It uses algorithms to analyze large datasets, improving performance as it is exposed to more data over time. Key applications include product recommendations, fraud detection, language translation, and optimizing supply chains. \nHow It Works\n1. Data Collection & Preprocessing:\nLarge datasets are collected and prepared for the algorithms. This involves cleaning the data (correcting errors, removing duplicates) and transforming it into a suitable, consistent format. \n2. Model Training:\nAlgorithms are trained on this data to identify patterns and relationships. The more data the model processes, the better it gets at its task. \n3. Model Output:\nThe result of training is a machine learning model that can be used to make predictions or classifications on new, 

In [36]:
quiz=response.get("quiz")

In [41]:
print(quiz) 


{'1': {'mcq': 'What is the main goal of machine learning?', 'options': {'a': 'To explicitly program systems', 'b': 'To learn from data and make decisions', 'c': 'To collect and preprocess data', 'd': 'To transform data into consistent format'}, 'correct': 'b'}, '2': {'mcq': 'Which type of machine learning uses labeled data for training?', 'options': {'a': 'Supervised Learning', 'b': 'Unsupervised Learning', 'c': 'Reinforcement Learning', 'd': 'Semi-supervised Learning'}, 'correct': 'a'}, '3': {'mcq': 'What is an example of a key application of machine learning in e-commerce?', 'options': {'a': 'Optimizing inventory levels', 'b': 'Predicting employee turnover', 'c': 'Recommendation engines for products', 'd': 'Identifying fraudulent transactions'}, 'correct': 'c'}, '4': {'mcq': 'What is the process of cleaning and transforming data before training the algorithms called?', 'options': {'a': 'Model Training', 'b': 'Data Collection', 'c': 'Data Preprocessing', 'd': 'Model Output'}, 'correct

In [42]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [43]:
quiz_table_data


[{'MCQ': 'What is the main goal of machine learning?',
  'Choices': 'a: To explicitly program systems | b: To learn from data and make decisions | c: To collect and preprocess data | d: To transform data into consistent format',
  'Correct': 'b'},
 {'MCQ': 'Which type of machine learning uses labeled data for training?',
  'Choices': 'a: Supervised Learning | b: Unsupervised Learning | c: Reinforcement Learning | d: Semi-supervised Learning',
  'Correct': 'a'},
 {'MCQ': 'What is an example of a key application of machine learning in e-commerce?',
  'Choices': 'a: Optimizing inventory levels | b: Predicting employee turnover | c: Recommendation engines for products | d: Identifying fraudulent transactions',
  'Correct': 'c'},
 {'MCQ': 'What is the process of cleaning and transforming data before training the algorithms called?',
  'Choices': 'a: Model Training | b: Data Collection | c: Data Preprocessing | d: Model Output',
  'Correct': 'c'},
 {'MCQ': 'Why is machine learning important 

In [45]:
quiz=pd.DataFrame(quiz_table_data)
quiz


,MCQ,Choices,Correct
0,What is the main goal of machine learning?,a: To explicitly program systems | b: To learn...,b
1,Which type of machine learning uses labeled da...,a: Supervised Learning | b: Unsupervised Learn...,a
2,What is an example of a key application of mac...,a: Optimizing inventory levels | b: Predicting...,c
3,What is the process of cleaning and transformi...,a: Model Training | b: Data Collection | c: Da...,c
4,Why is machine learning important for industries?,a: To make decisions without data | b: To gain...,b


In [46]:
quiz.to_csv("machinelearning.csv",index=False)


In [47]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'09_04_2025_04_13_36'